In [36]:
import pandas as pd
listings = pd.read_csv('listings.csv.gz')

In [37]:
def wrangle(df):

    df = df.copy()

    
    ''' Casting to new formats '''
    
    
    # As cost is in yen, drop the $ sign and convert all to numeric
    df['price'] = df['price'].str.replace('$','').str.replace(',','').astype(float)
    
    # Cast security deposit as floats. Replace the small amount of nulls with zero as zero accounts for half 
    # of all security deposits. Similarily clean the cleaning_fee column and extra_people
    df['security_deposit'] = df['security_deposit'].fillna('0').str.replace('$','').str.replace(',','').astype(float)
    df['cleaning_fee'] = df['cleaning_fee'].fillna('0').str.replace('$','').str.replace(',','').astype(float)
    df['extra_people'] = df['extra_people'].str.replace('$','').str.replace(',','').astype(float)

    # Keep this categorical feature but only those neighbourhoods with a higher frequency
    most_popular_neighbourhoods = df['neighbourhood_cleansed'].value_counts()[:23].index.tolist()
    df = df[df['neighbourhood_cleansed'].isin(most_popular_neighbourhoods)]

    # These columns are in string format but have innate hierarchy that could be useful 
    df['cancellation_policy'] = df['cancellation_policy'].map(
        {'flexible': 0, 'moderate': 1, 'strict_14_with_grace_period': 2, 'strict': 2, 'super_strict_30': 3, 
         'super_strict_60': 4})

    df['host_response_time'] = df['host_response_time'].fillna('within a day').map(
        {'within an hour': 1, 'within a few hours': 2, 'within a day': 3, 'a few days or more': 4})
    
    df['host_is_superhost'] = df['host_is_superhost'].map({'t': 1,'f': 0})

    df['instant_bookable'] = df['instant_bookable'].map({'t':1, 'f':0})
    
    
    
    ''' Fill or drop null values '''
    
    #Fill nulls
    df['host_response_rate'] = df['host_response_rate'].str.replace('%','').astype(float).fillna(100)
    df['host_identity_verified'] = df['host_identity_verified'].fillna('f').map({'t':1, 'f':0})
    
    # Drop small amount of nulls from columns
    df = df[df['review_scores_rating'].notnull()]
    df = df[df['bedrooms'].notnull()]
    df = df[df['beds'].notnull()]
    df = df[df['first_review'].notnull()]
    df = df[df['last_review'].notnull()]
    df['host_since'] = df['host_since'].dropna()
    
    
    
    ''' Engineer new features '''
    
    # Get the neighbourhood competition's avg (median) price 
    df['median_competing_prices'] = df.groupby('neighbourhood_cleansed')['price'].transform('median')
    
    # Query for hosts residing in Tokyo and create new column for local (Tokyo) hosts
    tokyo_strings = ['Tokyo', 'Tōkyō-to', '東京', 'tokyo', 'TOKYO', 'Tokyo-to', 'Tokyo To', 'tokyo-to']
    df['local_host'] = df['host_location'].fillna('tokyo').str.contains('|'.join(tokyo_strings)).astype(int)
    
    # Create a new column that estimates the duration the airbnb has existed
    df['first_review'] = pd.to_datetime(df['first_review'])
    df['last_review'] = pd.to_datetime(df['last_review'])
    df['airbnb_duration'] = (df['last_review'] - df['first_review']).dt.days 

    # Create a new column estimating how long one has been a host
    df['host_since'] = pd.to_datetime(df['host_since'])
    df['host_duration'] = (df['last_review'] - df['host_since']).dt.days
    
    # Estimate next month's airbnb income based on current availability 
    # (cannot account for additional bookings or cancelling)
    df['income_estimate'] = df['price'] * (30 - df['availability_30']) 
    
    
    # Cleaner, useable version of amenities
    def beautiful_amenities(df):

        # strip excess characters from column, convert each row to a list
        df['amenities'] = df['amenities'].str.replace('"','').str.replace('{','').str.replace('}','').str.split(',')        

        # Create new columns for 69 most commonly found amenities (there's probably a cleaner way)
        df['hot_tub'] = [1 if 'Hot tub' in row else 0 for row in df['amenities']]
        df['wifi'] = [1 if 'Wifi' or 'Internet' in row else 0 for row in df['amenities']]
        df['ac'] = [1 if 'Air conditioning' in row else 0 for row in df['amenities']]
        df['kitchen'] = [1 if 'Kitchen' in row else 0 for row in df['amenities']]
        df['washer'] = [1 if 'Washer' in row else 0 for row in df['amenities']]
        df['tv'] = [1 if 'TV' in row else 0 for row in df['amenities']]
        df['intercom'] = [1 if 'Buzzer/wireless intercom' in row else 0 for row in df['amenities']]
        df['heating'] = [1 if 'Heating' in row else 0 for row in df['amenities']]
        df['kid_friendly'] = [1 if 'Family/kid friendly' in row else 0 for row in df['amenities']]
        df['dryer'] = [1 if 'Dryer' in row else 0 for row in df['amenities']]
        df['smoke_detector'] = [1 if 'Smoke detector' in row else 0 for row in df['amenities']]
        df['cm_detector'] = [1 if 'Carbon monoxide detector' in row else 0 for row in df['amenities']]
        df['fire_extinguisher'] = [1 if 'Fire extinguisher' in row else 0 for row in df['amenities']]
        df['essentials'] = [1 if 'Essentials' in row else 0 for row in df['amenities']]
        df['shampoo'] = [1 if 'Shampoo' in row else 0 for row in df['amenities']]
        df['hangers'] = [1 if 'Hangers' in row else 0 for row in df['amenities']]
        df['hair_dryer'] = [1 if 'Hair dryer' in row else 0 for row in df['amenities']]
        df['iron'] = [1 if 'Iron' in row else 0 for row in df['amenities']]
        df['workspace'] = [1 if 'Laptop friendly workspace' in row else 0 for row in df['amenities']]
        df['self_check_in'] = [1 if 'Self check-in' in row else 0 for row in df['amenities']]
        df['lockbox'] = [1 if 'Lockbox' in row else 0 for row in df['amenities']]
        df['dishware'] = [1 if 'Dishes and silverware' in row else 0 for row in df['amenities']]
        df['safety_card'] = [1 if 'Safety card' in row else 0 for row in df['amenities']]
        df['shades'] = [1 if 'Room-darkening shades' in row else 0 for row in df['amenities']]
        df['hot_water'] = [1 if 'Hot water' in row else 0 for row in df['amenities']]
        df['bed_linens'] = [1 if 'Bed linens' in row else 0 for row in df['amenities']]
        df['microwave'] = [1 if 'Microwave' in row else 0 for row in df['amenities']]
        df['coffee_maker'] = [1 if 'Coffee maker' in row else 0 for row in df['amenities']]
        df['refrigerator'] = [1 if 'Refrigerator' in row else 0 for row in df['amenities']]
        df['cooking_basics'] = [1 if 'Cooking basics' in row else 0 for row in df['amenities']]
        df['stove'] = [1 if 'Stove' in row else 0 for row in df['amenities']]
        df['patio_or_balcony'] = [1 if 'Patio or balcony' in row else 0 for row in df['amenities']]
        df['long_term'] = [1 if 'Long term stays allowed' in row else 0 for row in df['amenities']]
        df['no_stairs'] = [1 if 'No stairs or steps to enter' in row else 0 for row in df['amenities']]
        df['first_aid'] = [1 if 'First aid kit' in row else 0 for row in df['amenities']]
        df['fire_extinguisher'] = [1 if 'Fire extinguisher' in row else 0 for row in df['amenities']]
        df['twenty_four_hour'] = [1 if '24-hour check-in' in row else 0 for row in df['amenities']]
        df['private_entrance'] = [1 if 'Private entrance' in row else 0 for row in df['amenities']]
        df['luggage_dropoff'] = [1 if 'Luggage dropoff allowed' in row else 0 for row in df['amenities']]
        df['well_lit'] = [1 if 'Well-lit path to entrance' in row else 0 for row in df['amenities']]
        df['other'] = [1 if 'Other' in row else 0 for row in df['amenities']]
        df['paid_parking'] = [1 if 'Paid parking off premises' in row else 0 for row in df['amenities']]
        df['pets_allowed'] = [1 if 'Pets allowed' in row else 0 for row in df['amenities']]
        df['extra_pillow'] = [1 if 'Extra pillows and blankets' in row else 0 for row in df['amenities']]
        df['ethernet'] = [1 if 'Ethernet connection' in row else 0 for row in df['amenities']]
        df['wide_hallways'] = [1 if 'Wide hallways' in row else 0 for row in df['amenities']]
        df['flat_path'] = [1 if 'Flat path to guest entrance' in row else 0 for row in df['amenities']]
        df['wide_entrance'] = [1 if 'Wide entrance' in row else 0 for row in df['amenities']]
        df['extra_space'] = [1 if 'Extra space around bed' in row else 0 for row in df['amenities']]
        df['wide_clearance'] = [1 if 'Wide clearance to shower' in row else 0 for row in df['amenities']]
        df['extra_space'] = [1 if 'Extra space around bed' in row else 0 for row in df['amenities']]
        df['shower_head'] = [1 if 'Handheld shower head' in row else 0 for row in df['amenities']]
        df['bedroom_lock'] = [1 if 'Lock on bedroom door' in row else 0 for row in df['amenities']]
        df['greet'] = [1 if 'Host greets you' in row else 0 for row in df['amenities']]
        df['keypad'] = [1 if 'Key pad' in row else 0 for row in df['amenities']]
        df['crib'] = [1 if 'Pack ’n Play/travel crib' in row else 0 for row in df['amenities']]
        df['wheelchair'] = [1 if 'Wheelchair accessible' in row else 0 for row in df['amenities']]
        df['cleaning'] = [1 if 'Cleaning before checkout' in row else 0 for row in df['amenities']]
        df['kitchenette'] = [1 if 'Kitchenette' in row else 0 for row in df['amenities']]
        df['full_kitchen'] = [1 if 'Full kitchen' in row else 0 for row in df['amenities']]
        df['pocket_wifi'] = [1 if 'Pocket wifi' in row else 0 for row in df['amenities']]
        df['events'] = [1 if 'Suitable for events' in row else 0 for row in df['amenities']]
        df['smart_lock'] = [1 if 'Smart lock' in row else 0 for row in df['amenities']]
        df['private_living'] = [1 if 'Private living room' in row else 0 for row in df['amenities']]
        df['garden_backyard'] = [1 if 'Garden or backyard' in row else 0 for row in df['amenities']]
        df['smoking'] = [1 if 'Smoking allowed' in row else 0 for row in df['amenities']]
        df['oven'] = [1 if 'Oven' in row else 0 for row in df['amenities']]
        df['single_level'] = [1 if 'Single level home' in row else 0 for row in df['amenities']]
        df['crib'] = [1 if 'Crib' in row else 0 for row in df['amenities']]

        return df

    # Apply cleaned amenities function
    df = beautiful_amenities(df)   
    
    
    
    ''' Drop unnecessary columns '''
    
    
    # Columns to drop
    all_or_none = ['thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_id', 'host_url', 'host_name', 'has_availability', 'requires_license',
                   'neighbourhood_group_cleansed', 'weekly_price', 'monthly_price', 'listing_url', 'scrape_id', 
                   'last_scraped', 'experiences_offered']
    strings = ['neighborhood_overview', 'interaction', 'access', 'transit', 'notes', 'house_rules', 'description', 
               'space', 'summary', 'name', 'license', 'amenities']
    irrelevant = ['host_location', 'host_about', 'host_acceptance_rate', 'host_thumbnail_url', 'host_picture_url', 
                  'is_location_exact', 'calendar_updated', 'calendar_last_scraped']
    redundant = ['city', 'state','zipcode', 'market', 'smart_location', 'country_code', 'country', 
                 'minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 
                 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'host_neighbourhood', 
                 'bathrooms', 'bedrooms']
    null_or_uniform = ['jurisdiction_names', 'is_business_travel_ready', 'require_guest_profile_picture', 
                       'require_guest_phone_verification', 'calculated_host_listings_count_entire_homes', 
                       'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms', 
                       'square_feet', 'host_listings_count', 'host_total_listings_count', 'host_verifications', 
                       'street', 'neighbourhood','host_has_profile_pic', 'reviews_per_month', 'host_since',
                       'first_review', 'last_review']

    # Drop unnecessary columns
    df = df.drop(columns=all_or_none)
    df = df.drop(columns=strings)
    df = df.drop(columns=null_or_uniform)
    df = df.drop(columns=redundant)
    df = df.drop(columns=irrelevant)

    return df

In [38]:
listings = wrangle(listings)

In [39]:
pd.set_option('display.max_columns', None)
listings.describe(exclude='number')

,neighbourhood_cleansed,property_type,room_type,bed_type
count,11622,11622,11622,11622
unique,23,24,4,5
top,Shinjuku Ku,Apartment,Entire home/apt,Real Bed
freq,1992,7011,8085,11573
